# Taxi mobility challenge

Heavy traffic can cause noise and atmospheric pollution. Optimizing the transportation system can therefore help in
improving the quality of citizens’ lives, both by facilitating their mobility and ensuring their health. 

In this challenge we address this problem by estimating the pollution reduction when the taxicab fleet changes from combustion engine-powered vehicles to electric vehicles and by predicting when the next taxi pick up will happen. Additionally, we identify hot spots per time of the day that can help the taxi companies optimizing their taxi rides.

More specifically, we answer the following questions:
1. Calculate the potential for a yearly reduction in CO2 emissions, caused by the taxi cabs roaming without passengers. Assume that the taxicab fleet is changing at the rate of 15% per month (from combustion engine-powered vehicles to electric vehicles). Assume also that the average passenger vehicle emits about 404 grams of CO2 per mile.

2. Build a predictor for taxi drivers, predicting the next place a passenger will hail a cab.

3. Identify clusters of taxi cabs that you find being relevant from the taxi cab company point of view.

## Library import

In [ ]:
import pandas as pd
import numpy as np
import glob
import geopy.distance

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import  MultipleLocator, FormatStrFormatter
plt.rcParams["figure.figsize"] = [12, 8]
plt.rcParams["figure.dpi"] = 100

import copy
from matplotlib.colors import LogNorm

## Functions

In [ ]:
def split_data(df, ind):
    """
    This function splits a dataframe by provided indexes.
    
    Inputs:
    - df: the dataframe to be splitted
    - ind: the indexes that the dataframe should be splitted upon
    
    Output:
    List of dataframes splitted on the specified indexes
    """
    dfs = []
    j = 0
    for val in ind:
        if j == 0:
            temp = df.iloc[:val]
            dfs.append(temp)
        elif j == len(ind):
            temp = df.iloc[val]
            dfs.append(temp)        
        else:
            temp = df.iloc[ind[j-1]:val]
            dfs.append(temp)
        j += 1
    return dfs

## Data import

In [ ]:
# Read multiple files in a loop
path = r"C:\Users\MNIX22\Documents\Projects\PMI\cabspottingdata"
all_files = glob.glob(path + "/*.txt")

cabs = []

for filename in all_files:
    cab_df = pd.read_csv(filename, index_col=None, header=None, sep=' ')
    cab_df.columns = ["latitude", "longitude", "occupancy", "time"] # assign column names 

    cabs.append(cab_df)

## Data validity checks

In [ ]:
cabs[0].describe()

In [ ]:
cabs[0].isna().any()

## 1. CO2 emissions reduction

To calculate the CO2 emission reduction caused by taxi cabs roaming without passengers, the first step is to calculate the time that each cab drives with no occupancy.

To do so, we assume that if there are no data from 15 minutes or more, then the taxi is "on a break". This means that we separate taxis' trajectories based on that assumption.

In [ ]:
# Calculate miles per cab
miles_per_cab = []
for i in range(len(cabs)):
    total_miles = 0
    cabs_temp = cabs[i].copy()  

    # Variable creation
    cabs_temp["date_time"] = pd.to_datetime(cabs_temp["time"],unit='s').dt.tz_localize("UTC").dt.tz_convert("America/Los_Angeles")
    cabs_temp["date_time_diff"] = cabs_temp["date_time"].diff()
    cabs_temp["coordinates"] = list(zip(cabs_temp.latitude, cabs_temp.longitude))
    cabs_temp["occupancy_change"] = cabs_temp["occupancy"].diff()
    cabs_temp["ind"] = cabs_temp.index

    # Split data on single "ride"
    # If there are no data in 15 minute and more then assume the taxi is not "active"
    route_ind = cabs_temp[cabs_temp["date_time_diff"].dt.total_seconds()/60 >= 15].index.values.tolist()
    route_ind.append(cabs_temp.index[-1])
    route_data = split_data(cabs_temp, route_ind)

    # Split data with no occupancy
    for single_route in range(len(route_data)):
        no_occ = route_data[single_route][route_data[single_route]["occupancy"] == 0].copy()
        if len(no_occ) > 0:
            no_occ["ind_diff"] = no_occ["ind"].diff()
            occupancy_index = no_occ[no_occ["ind_diff"] > 1].index.values.tolist()
            occupancy_index.append(no_occ.index[-1])
            occupancy_data = split_data(no_occ, occupancy_index)
            
            # Travelled miles no occupancy
            for miles in range(len(occupancy_data)):
                if len(occupancy_data[miles]) > 0:
                    coords_1 = occupancy_data[miles]["coordinates"].iloc[0]
                    coords_2 = occupancy_data[miles]["coordinates"].iloc[-1]
                    total_miles = total_miles + geopy.distance.geodesic(coords_1, coords_2).miles

    miles_per_cab.append(total_miles)

In [ ]:
# Calculate emissions based on cabs change
emissions = 0
for months in range(12):
    emissions = emissions + 0.85**months*len(cabs)*np.mean(miles_per_cab)*404
emissions

In [ ]:
# Calculate the difference of emissions
12*len(cabs)*np.mean(miles_per_cab)*404 - emissions

## 2. Pick up point prediction

### Exploratory data analysis

In [ ]:
# Number of fares per cab

fares = []
fares_df = []
for cab in range(len(cabs)):
    cabs_test = cabs[cab].copy()   
    cabs_test["occupancy_change"] = cabs_test["occupancy"].diff()
    fares.append(len(cabs_test[cabs_test["occupancy_change"] == 1]))
    fares_df.append(cabs_test[cabs_test["occupancy_change"] == 1])
    
sns.distplot(fares, kde=False, color = "blue", bins=20)
plt.xlabel("Number of fares per cab")

We see that most of the cabs have ~1000 fares within a month. There are just a few cabs that have more than 1200 fares in a month.

In [ ]:
# Nr of fares per weekday
cabs_test = pd.concat(fares_df) 
cabs_test["date_time"] = pd.to_datetime(cabs_test["time"],unit="s").dt.tz_localize("UTC").dt.tz_convert("America/Los_Angeles")
cabs_test["weekday"] = cabs_test["date_time"].dt.day_name()

sns.countplot(cabs_test["weekday"])
plt.xticks(fontsize=8)
plt.xlabel("Day of the week")
plt.show()

We see that most of the fares occur during the weekend. Nothing surprising here since it is connected to the people having free time and going out for entertainment.

In [ ]:
# Nr of fares per hour
cabs_test["hour"] = cabs_test["date_time"].dt.hour

sns.countplot(cabs_test["hour"])
plt.gca().xaxis.set_major_locator(MultipleLocator(10))
plt.gca().xaxis.set_major_formatter(FormatStrFormatter('%d'))
plt.xlabel("Hour of the Day")
plt.show()

We see that most of the fares occur during the nighttime hours. Nothing surprising here since it is connected to the people going out for entertainment.

In [ ]:
# Nr of fares per day and time
# Retructuring data
restr_data = cabs_test.groupby(["hour", "weekday"]).sum()["occupancy"].unstack()

## Reordering columns
restr_data = restr_data[['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']]
sns.heatmap(restr_data, cmap="Reds")

We see that most of the fares occur during the Friday and Saturday nighttime hours. Very few fares occur in the early hours, while we see some occurrence during the work commuting hours.

In [ ]:
# Duration of "idle" time
dur = []
for cab in range(len(cabs)):
    cabs_test = cabs[cab].copy()
    
    if cabs_test["occupancy"][0] == 1:
        cabs_test = cabs_test.iloc[1:]
    
    cabs_test["occupancy_change"] = cabs_test["occupancy"].diff()
    cabs_test["date_time"] = pd.to_datetime(cabs_test["time"],unit="s").dt.tz_localize("UTC").dt.tz_convert("America/Los_Angeles")
    temp = cabs_test[cabs_test["occupancy_change"] != 0].copy()
    temp["duration"] = temp["date_time"].diff()
    dur.append(temp[temp["occupancy_change"] == -1]["duration"])
    
duration = pd.concat(dur) / np.timedelta64(1, "m")

sns.distplot(duration, kde=False, color="blue")
plt.xlabel("Duration of idle time in minutes")

In [ ]:
sorted(duration, reverse = True)

In [ ]:
# Duration of trip time
dur = []
for cab in range(len(cabs)):
    cabs_test = cabs[cab].copy()
    
    if cabs_test["occupancy"][0] == 1:
        cabs_test = cabs_test.iloc[1:]
    
    cabs_test["occupancy_change"] = cabs_test["occupancy"].diff()
    cabs_test["date_time"] = pd.to_datetime(cabs_test["time"],unit="s").dt.tz_localize("UTC").dt.tz_convert("America/Los_Angeles")
    temp = cabs_test[cabs_test["occupancy_change"] != 0].copy()
    temp["duration"] = temp["date_time"].diff()
    dur.append(temp[temp["occupancy_change"] == 1]["duration"])
    
duration = pd.concat(dur) / np.timedelta64(1, "m")

sns.distplot(duration, kde=False, color="blue")
plt.xlabel("Duration of trips in minutes")

In [ ]:
sorted(duration, reverse = True)

### Data preparation

Create the necessary variables for the model.

In [ ]:
rides = []
for cab in range(len(cabs)):
    cabs_test = cabs[cab].copy()
    
    if cabs_test["occupancy"][0] == 1:
        cabs_test = cabs_test.iloc[1:]
        
    cabs_test["date_time"] = pd.to_datetime(cabs_test["time"],unit="s").dt.tz_localize("UTC").dt.tz_convert("America/Los_Angeles")
    cabs_test["hour"] = cabs_test["date_time"].dt.hour

    cabs_test["occupancy_change"] = cabs_test["occupancy"].diff()
    cabs_temp = cabs_test[cabs_test["occupancy_change"] != 0].copy()
    cabs_temp["duration"] = cabs_temp["date_time"].diff() / np.timedelta64(1, "m")
    # delete trip if > 180 minutes
    if len(cabs_temp[(cabs_temp.duration > 50) & ("occupancy_change" == -1)].index) > 0:
        idx = cabs_temp[(cabs_temp.duration > 50)].index
        idx_1 = idx - 1
        idx.append(idx_1)
        cabs_temp.drop(idx, inplace=True)

    cabs_temp.drop(["time", "occupancy_change"], axis=1, inplace = True)
    temp = pd.concat([cabs_temp, cabs_temp["latitude"].shift(), cabs_temp["longitude"].shift(), cabs_temp["latitude"].shift(2), cabs_temp["longitude"].shift(2), cabs_temp["latitude"].shift(3), cabs_temp["longitude"].shift(3), cabs_temp["latitude"].shift(4), cabs_temp["longitude"].shift(4)], axis=1).copy()
    temp.columns = ["latitude", "longitude", "occupancy", "date_time", "day", "hour", "weekday", "duration", 'latitude_1', 'longitude_1', 'latitude_2', 'longitude_2', 'latitude_3', 'longitude_3', 'latitude_4', 'longitude_4']

    rides.append(temp) #rides per cab

In [ ]:
all_rides = pd.concat(rides)
pick_up = all_rides[all_rides["occupancy"] == 1].copy()
pick_up.dropna(inplace = True)

In [ ]:
X = pick_up.drop(["latitude", "longitude", "occupancy", "date_time", "duration"], axis = 1)
y = pick_up[["latitude","longitude"]].copy()

Create train and test data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [ ]:
# random forest for multioutput regression
from sklearn.ensemble import RandomForestRegressor
# define model
model = RandomForestRegressor(n_estimators=100, random_state=1)
# fit model
model.fit(X_train, y_train)

In [ ]:
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

In [ ]:
print("Mean Square Error on training Data:{}".format(mean_squared_error(y_train, y_train_pred)))
print("Mean Square Error on testing Data:{}".format(mean_squared_error(y_test, y_test_pred)))

In [ ]:
plt.barh(X_train.columns, model.feature_importances_)

In [ ]:
y_test["coordinates"] = list(zip(y_test.latitude, y_test.longitude))
pred = pd.DataFrame(data = y_test_pred, columns=["latitude", "longitude"])
pred["coordinates"] = list(zip(pred.latitude, pred.longitude))

distance_error = []
for i in range(len(pred)):
    distance_error.append(geopy.distance.geodesic(y_test["coordinates"].iloc[i], pred["coordinates"].iloc[i]).miles)

In [ ]:
sns.distplot(distance_error, kde=False, color="blue")
plt.xlabel("Distance error in miles")

In [ ]:
print("Distance error in miles:{}".format(np.mean(distance_error)))